# 必要な準備を行う

実験中のワークフロー実行のために必要な準備を行います。  
上から順番に実行してください。

## 1. 開始する実験の実験パッケージ名を入力する

以下のセルを実行し、表示されるテキストボックスに作成したい実験パッケージ名を半角英数字で入力してください。  
データガバナンス機能にすでに存在するリポジトリ名のリポジトリは作成いただけません。実験名など分かりやすい名前をご記入ください。   
※入力値に誤りがある場合、もう一度実行することで訂正ができます。

In [ ]:
import os
import re
from IPython.display import clear_output

# GINサーバのものに合わせたバリデーションルールを設定
validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

print('作成したい実験パッケージ名を半角英数字で入力してください。')
while True:
    experiment_title = input("パッケージ名：")
    if validation.fullmatch(experiment_title):
        break
    else:
        clear_output()
        print('パッケージ名は英数字、および"-", "_", "."のみで入力してください。')

clear_output()

# 実験中のワークフローに該当実験パッケージを特定させるため、環境変数EXPERIMENT_TITLEに実験パッケージ名を設定
EXPERIMENT_TITLE = experiment_title
%store EXPERIMENT_TITLE
clear_output()

print("作成したパッケージ名：", experiment_title)
print('この実験名で処理を進めます。変更したい場合は、このセルをもう一度実行することで訂正ができます。')

In [ ]:
import json
import os

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

## 2. 実験記録管理のための準備を行う

### - 2.1 実験パッケージを用意する

In [ ]:
# このコンテナで扱う実験パッケージのパスを作成する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# 実験パッケージ名のフォルダが無ければ作成する
import os
os.chdir('/home/jovyan')
!mkdir -p $experiment_path

In [ ]:
import os

os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をパラメタファイルから取得する
monitoring_params = {}
with open(utils.fetch_monitoring_param_file_path(), mode='r') as f:
    monitoring_params = json.load(f)
    
scheme_name = monitoring_params['datasetStructure']

# urlを生成する
%cd ~/
remote_http_url = !git config --get remote.origin.url
for item in remote_http_url:
    remote_http_url = item

# 実験パッケージを用意する
!cp -r ~/WORKFLOWS/PACKAGE/scheme/$scheme_name/. $experiment_path
!cp -r ~/WORKFLOWS/PACKAGE/base/. $experiment_path

### - 2.2 pipeline.jsonに実験名を追記する

In [ ]:
import json

with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

pipeline.append(experiment_title)

with open('/home/jovyan/pipeline.json', 'w') as f:
    json.dump(pipeline, f, indent = 4)


## 3. ユーザー認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行行し、画面の表示に沿ってデータガバナンス機能に登録したユーザー名、パスワード、メールアドレスを入力してください。

In [ ]:
%cd ~/WORKFLOWS/EX-WORKFLOWS/util/scripts
import json
from scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    email = input("メールアドレス：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    tokens = response.json()

    # 既存のトークンがなければ作成する
    if len(tokens) < 1:
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))

    if response.status_code == HTTPStatus.OK or HTTPStatus.CREATED:
        tokens = response.json()
        clear_output()
        print("認証が正常に完了しました。次の手順へお進みください。")
        break
    else:
        clear_output()
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")

!git config --global user.name $name
!git config --global user.email $email

## 4. データ同期のための設定をする

この手順では、今の実行環境とデータガバナンス機能のリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

※実データの保存先として、必要であればAWS S3準拠のオブジェクトストレージを利用することもできます。  
もし当該ストレージを利用する場合は、このNotebookと合わせて[こちら](operate_S3_annex.ipynb)も実行してください。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json
from scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

# コンテナを消す際にコンテナとつなぐための公開鍵も削除のため、
# パラメータとしてGINから発行された鍵IDを保存
if response.status_code == HTTPStatus.CREATED:
    # params.jsonへの追記（鍵ID）
    params['ginKeyId'] = str(response.json()['id'])
    with open(utils.fetch_param_file_path(), mode='w') as f:
        json.dump(params, f, indent=4)
    print('Public key is ready.')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('Public key is ready before time.')

In [ ]:
from datalad import api
from IPython.display import clear_output

# sibling url をsshに変更する
%cd ~/
http_url = !git config --get remote.origin.url
for item in http_url:
    http_url = item
    ssh_url = item.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
    
# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
print('SSH connection is ready.')

## 5. READMEに実験実行環境へのリンクを追加する

当実行環境へアクセスするためのリンクを、データガバナンス機能の当研究リポジトリのREADMEに追記します。

In [ ]:
import os, urllib
from IPython.display import clear_output
from scripts import utils

print('当画面のURLをブラウザからコピーし、以下のフォームに入力してください。')
url = input()
print(url.find("/notebooks/"))
url = url[:url.find("/notebooks/")] + "/notebooks/WORKFLOWS/experiment.ipynb"

with open(experiment_path + '/README.md', 'w', newline='\n') as f:
    f.write("\n## 実験実行環境にアクセスしたい場合\n以下のリンクをクリックしてください<br>" + url + "  ")
    f.write("\n\n上記リンクからアクセスできない場合は以下のリンクから実験実行環境を再度立ち上げてください  ")
    f.write("\n※「他リポジトリから実験の入力データを用意する」で入力データを用意した場合は、再度必要なファイルの実体をdatalad get　'ファイルパス'で取得する必要があります。  ")
    f.write("\nhttps://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(http_url, safe='') + "/HEAD?filepath=WORKFLOWS/experiment.ipynb")

clear_output()
print('READMEに実行環境へのリンクを追加しました。')

## 6. 実行結果を研究リポジトリに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%cd ~/WORKFLOWS/EX-WORKFLOWS/util/scripts
from scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

In [ ]:
import os
import glob
import json

# Git管理のパスのリストを作成する
%cd $experiment_path
files = os.listdir()

# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')

# 実験パッケージ直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]

# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)

# Git-annex管理するパスの配列を作成する
annexed_save_path = [experiment_path + '/input_data', experiment_path + '/output_data']

In [ ]:
import papermill as pm

%cd ~/
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_ワークフロー実行準備 (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False)
)

In [ ]:
import papermill as pm

%cd ~/
# Git管理ファイルを保存
pm.execute_notebook(
    'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_ワークフロー実行準備 (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
)

## 7. ワークフロー図を更新する

ワークフロー図にこのワークフローが実行済みであることを反映します。

In [ ]:
%cd ~/
path = 'WORKFLOWS/EX-WORKFLOWS/images/notebooks.diag'

with open('.gitignore', 'r') as f:
    text = f.read()
    if text.find(path) == -1:
        !echo "/" + $path >> ./.gitignore

# notebooks.diagのgit管理を外す
!git update-index --skip-worktree $path

In [ ]:
%cd ~/
find = '"required_every_time"[fontsize = 14];'
replace = '"required_every_time"[numbered = 済, fontsize = 14];'

with open(path, 'r') as f:
    s = f.read()

with open(path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

## 8. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../../experiment.ipynb)からトップページに遷移できます。  